In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from src import eda, util, fe, models
import ipywidgets as widgets

# ---- EDA Konfiguration ----
# Skalierung aller Sensoren und Operation Settings (Z-Norm pro Betriebsbedingung)
NORMALIZE = True

EDA_FD001 = True
EDA_FD002 = True
EDA_FD003 = True
EDA_FD004 = True

# ---- Caching-/Recompute-Konfiguration ----
PRECOMPUTE_ALL_PLOTS = False        # Alle Plots bei Notebook-Start berechnen. Widgets reagieren erst nach dem Precompute.
FORCE_RECOMPUTE_PLOTS = False       # Alle Plots beim Precompute immer neu berechnen, auch wenn sie schon im Cache sind.
FORCE_RECOMPUTE_TSNE_DBSCAN = False # TSNE/DBSCAN immer neu berechnen

if not PRECOMPUTE_ALL_PLOTS:        # Nur gültig, wenn Precompute aktiviert ist!
    FORCE_RECOMPUTE_PLOTS = False   
    FORCE_RECOMPUTE_TSNE_DBSCAN = False

<table style="width:100%; background-color: white; padding: 10px; border-radius: 6px; box-shadow: 0 0 5px rgba(0,0,0,0.2);">
  <tr>
    <td>
      <h1 style="margin-bottom: 0; color: black; font-size: clamp(1.4rem, 2.2vw, 2.2rem);">
        Predictive Maintenance: Vorhersage der Remaining Useful Life (RUL) von Triebwerken des NASA C-MAPSS-Datensatzes
      </h1>
    </td>
    <td align="right">
      <img src="images/OST_Logo_DE_RGB@2000ppi.png" alt="OST Logo" width="200">
    </td>
  </tr>
</table>

**Autor:** Rino Albertin  
**Datum:** 6. März 2025

---
## Inhaltsverzeichnis

1. Einleitung  
   1.1 Zielsetzung  
   1.2 Vorgehensweise  

2. Überblick über den Datensatz und technische Grundlagen  
   2.1 Allgemeine Datenstruktur  
   2.2 Operative Einstellungen und Sensorik  

3. Einzelanalyse  
   3.1 FD001  
   3.2 FD002  
   3.3 FD003  
   3.4 FD004  

4. Kombinierte Analyse aller Datensätze  
   4.1 Explorative Datenanalyse  
   4.2 Feature Engineering  
   4.3 Modellierung  
   4.4 Hyperparameter-Tuning  
   4.5 Evaluation  

5. Vergleich der Modellresultate  

6. Fazit und Ausblick

# Anhang

---
## 1. Einleitung

Predictive Maintenance ist in der heutigen Industrie von zentraler Bedeutung, um ungeplante Ausfälle zu vermeiden und Wartungskosten zu senken. Die Vorhersage der verbleibenden Nutzungsdauer (Remaining Useful Life, RUL) von Triebwerken ermoeglicht eine optimierte Planung von Wartungsarbeiten und verbessert die Betriebseffizienz.

Der NASA (C-MAPSS) Commercial Modular Aero-Propulsion System Simulation-Datensatz bietet simulierte Run-to-Failure-Zeitreihen von Triebwerken der NASA, die Zeit, Betriebsbedingungen, 3 operative Einstellungen und 21 Sensormessungen umfassen. Dies macht ihn zu einer idealen Grundlage, um moderne Machine-Learning-Methoden zur RUL-Vorhersage zu evaluieren und zu vergleichen.

## 1.1 Zielsetzung

Entwicklung eines Machine-Learning-Modells, das auf Basis historischer Sensordaten die RUL von Triebwerken präzise vorhersagen kann. Dabei soll sowohl das Verhalten einzelner Szenarien als auch eine generalisierbare Lösung über alle vier Datensätze hinweg untersucht werden.


## 1.2 Vorgehensweise

Die Bearbeitung erfolgt in zwei aufeinander aufbauenden Phasen:

### Phase 1: Einzelanalysen pro Datensatz (FD001–FD004)
Jeder der vier C-MAPSS-Datensätze wird separat analysiert, um spezifische Charakteristika und Herausforderungen zu verstehen. Für jeden Datensatz wird dieselbe Pipeline angewendet:

1. **Explorative Datenanalyse (EDA):**
   - Visualisierung typischer Sensorverläufe.
   - Analyse der Datenstruktur und Lebensdauerverläufe.

2. **Feature Engineering:**
   - Selektion und Transformation relevanter Sensoren.
   - Umgang mit korrelierten oder konstanten Features.

3. **Modellierung:**
   - Auswahl geeigneter Machine-Learning-Modelle.
   - Aufbau einer Pipeline (Preprocessing, Training, Inferenz).

4. **Hyperparameter-Tuning:**
   - Optimierung mit `GridSearchCV` zur groben Parameterauswahl.
   - Erweiterte Feinabstimmung vielversprechender Modelle mit `RandomizedSearchCV`.

5. **Evaluation:**
   - Validierung.
   - Vergleich der Szenarien und Modelle.

### Phase 2: Kombinierte Analyse (Generalmodell)
In Phase 2 werden die vier Datensätze zusammengeführt, um ein Modell zu entwickeln, das über unterschiedliche Szenarien hinweg verallgemeinerbar ist. Die Analyse basiert auf der gleichen Pipeline wie in Phase 1.

**Referenzen:**  
- NASA C-MAPSS-Datensatz: https://data.nasa.gov/d/ff5v-kuh6, Alternativ verfügbar unter: [Kaggle – NASA Turbofan Jet Engine Data Set](https://www.kaggle.com/datasets/behrad3d/nasa-cmaps)
- Machine Learning Unterichtsunterlagen der OST – Ostschweizer Fachhochschule
---

## 2. Überblick über den Datensatz und technische Grundlagen

Bevor mit der eigentlichen Analyse begonnen wird, erfolgt in diesem Kapitel eine technische Einführung in den C-MAPSS-Datensatz. Ziel ist es, ein grundlegendes Verständnis für Struktur, Sensorik und die inhaltlichen Rahmenbedingungen zu schaffen. Eine erste übergreifende EDA bietet zusätzlich einen Einblick in Gemeinsamkeiten und Besonderheiten der vier Szenarien.

### 2.1 Allgemeine Datenstruktur

Der C-MAPSS-Datensatz (Commercial Modular Aero-Propulsion System Simulation) besteht aus vier Teildatensätzen (FD001–FD004), die unterschiedliche Betriebsszenarien simulieren. Jeder Datensatz enthält Sensormessungen von mehreren Triebwerken über deren gesamte Lebensdauer hinweg (Run-to-Failure).

Für jede Zeile sind folgende Informationen verfügbar:

- **unit**: ID des Triebwerks (eine Einheit)
- **time**: aktueller Zyklus (Zeitpunkt)
- **op_setting_1–3**: operative Einstellungen (Flughöhe, Machzahl, Drosselklappen-Positionen)
- **sensor_1–21**: Sensormessungen aus verschiedenen Triebwerkskomponenten

![Triebwerksaufbau in C-MAPSS](images/Simplified%20diagram%20of%20engine%20simulated%20in%20C-MAPSS.png)

Die Abbildung zeigt den schematischen Aufbau eines Triebwerks, wie es in der C-MAPSS-Simulation verwendet wird. Die Hauptkomponenten sind:

- **Fan:** Der Fan (Luftgebläse) saugt Umgebungsluft an, die teilweise in den Bypass strömt (äusserer Luftstrom) und teilweise ins Triebwerk.
- **LPC (Low Pressure Compressor):** Komprimiert die angesaugte Luft bei niedrigem Druck.
- **HPC (High Pressure Compressor):** Erhöht den Druck der Luft weiter vor der Verbrennung.
- **Combustor:** Vermischt die verdichtete Luft mit Treibstoff, zündet die Mischung und erzeugt damit heisse Hochdruckgase.
- **HPT (High Pressure Turbine):** Entzieht den heissen Gasen Energie, um den HPC anzutreiben.
- **LPT (Low Pressure Turbine):** Treibt den Fan und die LPC an.
- **N1/N2:** Repräsentieren die beiden Hauptwellen im Triebwerk (N1: Fan + LPC, N2: HPC + HPT).
- **Nozzle:** Düse am Austritt – beschleunigt die Abgasströmung und erzeugt Schub.

Viele der 21 Sensoren messen Parameter an genau diesen Stellen, z. B. Druck, Temperatur, Drehzahl oder Luftmassenströme.

### 2.2 Operative Einstellungen und Sensorik

Die C-MAPSS-Daten umfassen neben Zyklusinformationen auch drei operative Einstellungen und 21 Sensormessungen. Die operativen Einstellungen variieren je nach Szenario und beeinflussen die physikalischen Messgrössen, die an verschiedenen Triebwerkskomponenten aufgezeichnet werden.

#### Operative Einstellungen

| Spalte         | Beschreibung                         | Wertebereich        |
|----------------|--------------------------------------|---------------------|
| op_setting_1   | Flughöhe (Altitude)                  | 0 – 42'000 ft       |
| op_setting_2   | Machzahl                             | 0 – 0.84            |
| op_setting_3   | Throttle Resolver Angle (TRA)        | 20 – 100            |

#### Sensorübersicht

| Sensor | Beschreibung                             | Wertebereich (FD001–FD004) |
|--------|------------------------------------------|----------------------------|
|  1     | Total temperature at fan inlet (T2)      | 445.0 – 518.67 °R          |
|  2     | Total temperature at LPC outlet (T24)    | 535.48 – 645.11 °R         |
|  3     | Total temperature at HPC outlet (T30)    | 1242.67 – 1616.91 °R       |
|  4     | Total temperature at LPT outlet (T50)    | 1023.77 – 1441.49 °R       |
|  5     | Pressure at fan inlet (P2)               | 3.91 – 14.62 psia          |
|  6     | Bypass-duct pressure (P15)               | 5.67 – 21.61 psia          |
|  7     | Total pressure at HPC outlet (P30)       | 136.17 – 570.81 psia       |
|  8     | Physical fan speed (Nf)                  | 1914.72 – 2388.64 rpm      |
|  9     | Physical core speed (Nc)                 | 7984.51 – 9244.59 rpm      |
| 10     | Engine pressure ratio (epr)              | 0.93 – 1.32                |
| 11     | Static pressure at HPC outlet (Ps30)     | 36.04 – 48.53 psia         |
| 12     | Ratio of fuel flow to Ps30 (phi)         | 128.31 – 537.49 pps/psi    |
| 13     | Corrected fan speed (NRf)                | 2027.57 – 2390.49 rpm      |
| 14     | Corrected core speed (NRc)               | 7845.78 – 8293.72 rpm      |
| 15     | Bypass ratio (BPR)                       | 8.1563 – 11.0669           |
| 16     | Burner fuel-air ratio (farB)             | 0.02 – 0.03                |
| 17     | Bleed enthalpy (htBleed)                 | 302   – 400                |
| 18     | Demanded fan speed (Nf_dmd)              | 1915   – 2388 rpm          |
| 19     | Demanded corrected fan speed (PCNfR_dmd) | 84.93 – 100.0 rpm          |
| 20     | HPT coolant bleed (W31)                  | 10.16 – 39.89 lbm/s        |
| 21     | LPT coolant bleed (W32)                  | 6.0105 – 23.9505 lbm/s     |

*Die angegebenen Wertebereiche wurden über alle Trainingsdaten der Szenarien FD001 bis FD004 bestimmt.*

In [ ]:
datasets = {
    "FD001": util.load_cmapss_data("FD001"),
    "FD002": util.load_cmapss_data("FD002"),
    "FD003": util.load_cmapss_data("FD003"),
    "FD004": util.load_cmapss_data("FD004"),
    "ALL":   util.load_cmapss_data()
}

raw_datasets = {k: (df_train.copy(), df_test.copy()) for k, (df_train, df_test) in datasets.items()}

df_train_01, df_test_01 = datasets["FD001"]
df_train_02, df_test_02 = datasets["FD002"]
df_train_03, df_test_03 = datasets["FD003"]
df_train_04, df_test_04 = datasets["FD004"]
df_train, df_test = datasets["ALL"]

In [ ]:
df_train.head()

In [ ]:
df_train.describe()

In [ ]:
df_train.info()

### 2.3 Vorverarbeitung – Skalierung nach Betriebs­bedingungen

Um Niveauverschiebungen auszublenden, die allein durch wechselnde Flughöhen, Machzahlen oder Drosselklappen-Stellungen entstehen (op_settings), werden alle 21 Sensoren **z-standardisiert – getrennt pro Betriebs­gruppe (`op_cond`)**.

Die Gruppierung erfolgt anhand fester Schwellenwerte (siehe Plot unten) in einer **6 × 5 × 2-Einteilung**:

| Einstellgrösse    | Binning-Strategie                            | Zweck                           |
|-------------------|----------------------------------------------|----------------------------------|
| **op_setting 1**  | manuell: `[0 – 5 – 15 – 22 – 30 – 40 – ∞]`    | trennt Flughöhenbereiche        |
| **op_setting 2**  | manuell: `[0 – 0.2 – 0.55 – 0.63 – 0.8 – ∞]`  | trennt Geschwindigkeitsbereiche |
| **op_setting 3**  | binär: `< 80` → T0, `≥ 80` → T1               | Zweipunkt-Steuerung (TRA)       |

Diese Einteilung ergibt **60 theoretisch mögliche Kombinationen** (6 × 5 × 2).  
**Tatsächlich kommen aber in jedem Szenario nur genau 6 Gruppen vor** – ein Grossteil der Kombinationen tritt in den Daten nicht auf. Dadurch bleibt die Normalisierung robust und gut interpretierbar.

- Der `StandardScaler` wird **ausschliesslich auf den Trainingsdaten** pro `op_cond`-Gruppe fit-transformiert.
- Für die zugehörigen Testdaten wird **dieselbe Skalierung** mittels `transform()` übernommen.
- Bei **konstanten Betriebspunkten** (z. B. FD001 & FD003) degeneriert das Verfahren zu einer **globalen Standardisierung**, da nur eine einzige `op_cond`-Gruppe existiert.

> **Hinweis:** Alle tatsächlich vorkommenden Gruppen enthalten mindestens **5000 Zeilen**, was eine robuste, gruppenbasierte Skalierung erlaubt.

In [ ]:
util.plot_op_settings_histograms(df_train_02, df_test_04)

In [ ]:
if NORMALIZE:
    for k in datasets:
        datasets[k] = util.standardize_by_op_cond(*datasets[k])
else:
    for k in datasets:
        datasets[k] = tuple(util.assign_op_cond_bins(df) for df in datasets[k])

df_train_01, df_test_01 = datasets["FD001"]
df_train_02, df_test_02 = datasets["FD002"]
df_train_03, df_test_03 = datasets["FD003"]
df_train_04, df_test_04 = datasets["FD004"]
df_train, df_test = datasets["ALL"]

print("df_train_02:", util.get_op_cond_distribution_summary(df_train_02), "\n")
print("df_test_02:", util.get_op_cond_distribution_summary(df_test_02), "\n")

print("df_train_04:", util.get_op_cond_distribution_summary(df_train_04), "\n")
print("df_test_04:", util.get_op_cond_distribution_summary(df_test_04), "\n")

print("df_train:", util.get_op_cond_distribution_summary(df_train), "\n")
print("df_test:", util.get_op_cond_distribution_summary(df_test), "\n")

In [ ]:
df_train.head()

In [ ]:
df_train.describe()

---
## 3. Einzelanalysen der vier Datensätze

Ziel dieses Kapitels ist es, die vier Teildatensätze **FD001 bis FD004** separat zu analysieren und zu modellieren. Jeder Datensatz repräsentiert ein eigenes Betriebsszenario mit unterschiedlichen Rahmenbedingungen (konstante vs. variable Settings, ein oder zwei Degradationsmodi).

Für jeden Datensatz wird eine identische Analysepipeline angewendet, bestehend aus:
- Explorative Datenanalyse (EDA)
- Feature Engineering
- Modellierung
- Hyperparameter-Tuning
- Evaluation

Diese Einzelschritte ermöglichen ein besseres Verständnis der Stärken und Schwächen verschiedener Modelle je Szenario und legen die Basis für die spätere kombinierte Analyse in Kapitel 4.

---
### 3.1 Analyse für FD001
Der Datensatz FD001 stellt das einfachste Szenario innerhalb von C-MAPSS dar: konstante Betriebsbedingungen und ein einziger Degradationsmodus. Er eignet sich daher besonders gut für eine erste Modellierung und das Testen von Grundkonzepten.

#### 3.1.1 Explorative Datenanalyse (EDA)
Ziel dieses Abschnitts ist es, ein erstes Verständnis für die Struktur und Eigenschaften des Datensatzes FD001 zu entwickeln. Dabei werden typische Lebensdauerverläufe analysiert, exemplarische Sensorwerte visualisiert und erste Hinweise auf potenziell relevante Merkmale identifiziert.

In [ ]:
if EDA_FD001:
    # ─────────────────────────────
    # Konfiguration für FD001 EDA
    # ─────────────────────────────
    # Steuerung des PNG-Cachings pro Plot-Sektion.
    # Falls False: PNG wird gelöscht und neu erstellt.
    USE_CACHE = {
        "overview": True,
        "ops": True,
        "sensors": True,
        "cluster": True,
    }
    FORCE_RECOMPUTE_TSNE_DBSCAN = FORCE_RECOMPUTE_TSNE_DBSCAN
    # FORCE_RECOMPUTE_TSNE_DBSCAN = True
    # ─────────────────────────────

    toggle = util.make_toggle_shortcut(df_train_01, "FD001")
    sensor_cols = [f"sensor_{i}" for i in [2, 3, 4, 6, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21]]

    overview_plots = [
        toggle("1-1. Lebensdauerkennzahlen", eda.describe_life_stats),
        toggle("1-2. Lebensdauerverteilung", eda.plot_life_distribution),
    ]
    ops_plots = [
        toggle("2-1. Verläufe Operation Settings", eda.plot_opsetting_curves, unit_ids=[39, 69]),
        toggle("2-2. Korrelation Operation Settings", eda.plot_opsetting_correlation_matrix),
        toggle("2-3. Verteilung im letzten Zyklus", eda.plot_opsetting_box_violin_last_cycle),
        toggle("2-4. Verteilung nach Quantilen", eda.plot_opsetting_distributions_by_cycle_range, lower_quantile=0.25, upper_quantile=0.75),
        toggle("2-5. RUL-Korrelation", eda.plot_opsetting_rul_correlation),
        toggle("2-6. Trend normierte Zeit", eda.plot_average_opsetting_trend_normalized_time),
    ]
    sensors_plots = [
        toggle("3-1. Sensorverläufe", eda.plot_single_sensor_curves, unit_ids=range(1, 6), rolling_window=10),
        toggle("3-2. Sensor-Overlay", eda.plot_sensor_overlay, unit_id=69, dataset_name="FD001-69"),
        toggle("3-3. Sensor-Korrelation", lambda df: (
            (fig := plt.figure(figsize=(28, 10))),
            (axs := fig.subplots(1, 2)),
            eda.plot_sensor_correlation_matrix(df, dataset_name="FD001", ax=axs[0]),
            eda.plot_sensor_correlation_matrix(df, sensor_cols=sensor_cols, dataset_name="FD001 (ohne konstante)", annot=True, ax=axs[1]),
            plt.tight_layout(),
        )),
        toggle("3-4. Box/Violin letzter Zyklus", eda.plot_sensor_box_violin_last_cycle, sensor_cols=sensor_cols),
        toggle("3-5. Sensorverteilung nach Lebensdauer", eda.plot_sensor_distributions_by_cycle_range, sensor_cols=sensor_cols),
        toggle("3-6. Sensorverteilungen nach op_cond", eda.plot_sensor_distributions_by_cycle_range, hue_col="op_cond", sensor_cols=sensor_cols),
        toggle("3-7. RUL-Korrelation Sensoren", eda.plot_sensor_rul_correlation, sensor_cols=sensor_cols),
        toggle("3-8. Sensortrend normierte Zeit", eda.plot_average_sensor_trend_normalized_time, sensor_cols=sensor_cols),
    ]

    # Einstellungen von plot_tsne_dbscan_clusters
    toggle_tsne = widgets.Output()
    with toggle_tsne:
        fig, labels = eda.plot_tsne_dbscan_clusters(
            df_train_01,
            feature_cols = sensor_cols,
            dataset_name="FD001",
            force_recompute=FORCE_RECOMPUTE_TSNE_DBSCAN
        )
        df_train_01["cluster_tsne"] = labels

    cluster_plots = [
        toggle("4-1. TSNE + DBSCAN Cluster", eda.plot_tsne_dbscan_clusters), # plot_tsne_dbscan_clusters zeigt nur den Plot an
        toggle("4-2. op_settings je Cluster (Boxplot)", eda.plot_op_settings_vs_cluster, cluster_col="cluster_tsne"),
        toggle("4-3. Cluster-Transitions (Sankey)", lambda df: (
            fig := eda.plot_cluster_transitions_sankey(df, cluster_col="cluster_tsne", dataset_name="FD001"),
            fig.update_layout(width=1800, height=600),
            fig
        )),
        toggle("4-4. Durchschnittlicher Zeitpunkt je Cluster", eda.plot_cluster_average_time, cluster_col="cluster_tsne", dataset_name="FD001"),
        toggle("4-5. Clusterverteilung letzter Zyklus", eda.plot_cluster_distribution_last_cycle, cluster_col="cluster_tsne"),
        toggle("4-6. Lebensdauer pro finalem Cluster", eda.plot_lifetime_boxplot_by_cluster, cluster_col="cluster_tsne"),
        toggle("4-7. Mittlere Sensorwerte pro Cluster", eda.plot_mean_normalized_sensors_by_cluster, sensor_cols=sensor_cols, cluster_col="cluster_tsne"),
        toggle("4-8. Sensorverteilungen nach Cluster", eda.plot_sensor_distributions_by_cycle_range, hue_col="cluster_tsne", sensor_cols=sensor_cols),
        toggle("4-9. Trend Sensoren je Cluster",
               lambda df: util.make_cluster_navigation_panel(
                   df=df,
                   cluster_col="cluster_tsne",
                   cluster_plot_func=eda.plot_average_sensor_trend_normalized_time,
                   sensor_cols=sensor_cols,
                   dataset_name="FD001",
                   force_recompute=not USE_CACHE["cluster"]
               )),
        toggle("4-10. Cluster-Zusammenfassung (Tabelle)", eda.summarize_cluster_characteristics, cluster_col="cluster_tsne"),
    ]

    if PRECOMPUTE_ALL_PLOTS:
        util.cache_util.cache_all_plots(
            [overview_plots, ops_plots, sensors_plots, cluster_plots],
            dataset_name="FD001",
            force_recompute=FORCE_RECOMPUTE_PLOTS
        )

    sections = [
        util.make_dropdown_section(overview_plots, "FD001", use_cache=USE_CACHE["overview"]),
        util.make_dropdown_section(ops_plots, "FD001", use_cache=USE_CACHE["ops"]),
        util.make_dropdown_section(sensors_plots, "FD001", use_cache=USE_CACHE["sensors"]),
        util.make_dropdown_section(cluster_plots, "FD001", use_cache=USE_CACHE["cluster"]),
    ]
    tab_titles = [
        "1. Übersicht",
        "2. Operation Settings",
        "3. Sensoren",
        "4. Clusteranalyse",
    ]
    eda_panel_FD001 = util.make_lazy_panel_with_tabs(
        sections,
        tab_titles=tab_titles,
        open_btn_text="FD001 EDA öffnen",
        close_btn_text="Schliessen"
    )
    display(eda_panel_FD001)

**Lebensdauerverteilung:**  
Die Lebensdauer der Triebwerke reicht von 128 bis 362 Zyklen (Median: 199). Die Verteilung ist leicht rechtsschief und weist einige Ausreisser mit hoher Zyklenanzahl auf.

**Operation Settings:**  
Die Betriebsbedingungen sind in diesem Szenario konstant und nur leicht verrauscht. `op_setting_1` und `op_setting_2` zeigen minimale Streuung, während `op_setting_3` konstant bleibt. Die Korrelationen der Settings mit der RUL sind vernachlässigbar (alle < 0.01). Auch innerhalb der Lifetime-Cluster zeigen sich keine signifikanten Unterschiede, weshalb diese Settings nicht zur Modellierung herangezogen werden sollten.

**Sensoranalyse:**  
Die Sensoren lassen sich in mehrere Gruppen mit stark korrelierten Verläufen einteilen. Konstante Sensoren wurden zuvor entfernt:

- **Gruppe A:** `sensor_2`, `3`, `4`, `8`, `11`, `13`, `15`, `17`  
- **Gruppe B:** `sensor_7`, `12`, `20`, `21`  
- **Gruppe C:** `sensor_9`, `14`  
- **Gruppe D:** `sensor_6` – stark verrauscht, nicht informativ

Mehrere Sensoren zeigen klare Trends über die Zeit und starke Korrelationen mit der RUL. Sensorverläufe über normierte Zeit zeigen für viele Sensoren monotone, glatte Verläufe. Diese sind potenziell prädiktiv und für Feature Engineering geeignet. Die Verteilungen zeigen teils Unterschiede zwischen Units mit langer und kurzer Lebensdauer, was weitere Segmentierungen rechtfertigt. Die Verteilungen sind meist unimodal im letzen Zyklus und weissen wenig Aussreiser auf. Die Ausreisser wurden dabei nicht entfernt da sie potenziel wichitge Informationen liefern.

**Clusteranalyse:**  
Ein t-SNE/DBSCAN-Verfahren identifiziert zwei Cluster, wobei der kleinere (Cluster 1) ca. 2 % der Daten umfasst. Er enthält keine finalen Datenpunkte (also keinen Lebensdauer-Endpunkt). Der Eintritt in diesen Cluster erfolgt typischerweise deutlich früher (Ø = 0.255 normierte Zeit) als bei Cluster 0 (Ø = 0.507).

Sensorverläufe in Cluster 1 sind stark verrauscht und flach. In Cluster 0 hingegen zeigen sich klar strukturierte Abnahmen bzw. Zunahmen – konsistent mit dem Verschleissverhalten. Daraus ergibt sich:

- **Cluster 0**: Hauptcluster mit verwertbaren Mustern und finalen Datenpunkten  
- **Cluster 1**: Frühzeitiger Sondercluster ohne verwertbare Zielgrösse (RUL)

#### 3.1.2 Feature Engineering

Auf Basis der EDA wurden gezielt Sensoren ausgewählt, die für die Prognose der Restlebensdauer (RUL) eine hohe Aussagekraft besitzen. Die Auswahl erfolgte entlang physikalischer Kriterien, des zeitlichen Signalverlaufs, der Korrelation zur Zielgrösse sowie einer Redundanzanalyse innerhalb stark korrelierter Sensorgruppen. Konstant bleibende oder verrauschte Sensoren wurden entfernt.

Zur besseren Übersicht wurden die verbleibenden Sensoren in Gruppen eingeteilt und repräsentative Merkmale extrahiert. Zusätzlich wurden kombinierte Features gebildet, um komplexe physikalische Zusammenhänge wie thermodynamische Effizienz oder mechanische Wechselwirkungen modellierbar zu machen.

Um den physikalischen Kontext zu verdeutlichen, wurden im ersten Verarbeitungsschritt alle Sensoren und Betriebsbedingungen in Klartext umbenannt. Ein weiterer Verarbeitungsschritt bestand darin, das Trainingsset gezielt zu kürzen: Bei einem zufälligen Anteil von 25 % der Triebwerks-Units wurde das letzte Segment abgeschnitten, sodass diese Einheiten nicht bis zum vollständigen Ausfall beobachtet wurden. Dadurch wird das Modell gezwungen, auch aus unvollständigen Lebenszyklen zu lernen, was der realen Situation im laufenden Betrieb besser entspricht.

In [ ]:
np.random.seed(42)
df_train_01, df_test_01 = raw_datasets["FD001"]
df_train_01_fe = fe.rename_opsettings_and_sensors(df_train_01)
df_test_01_fe = fe.rename_opsettings_and_sensors(df_test_01)
df_train_01_fe = fe.truncate_train_units(df_train_01_fe)

##### Gruppe A: Temperatur, Druck, Drehzahl & Verhältnisse

Verwendet wurden `sensor_4_T50_LPT_outlet_temp`, `sensor_11_Ps30_HPC_static_pres`, `sensor_15_BPR_bypass_ratio` und `sensor_17_htBleed_bleed_enthalpy`, da sie zentrale thermodynamische und aerodynamische Zustände des Triebwerks abbilden. `sensor_4` zeigte die höchste Korrelation mit der RUL und ist physikalisch am sinnvollsten, da er die Temperatur ganz am Ende des Systems misst. Die Sensoren `sensor_2_T24` und `sensor_3_T30` wurden aufgrund starker Redundanz mit `sensor_4` ausgeschlossen. Ergänzend wurden mehrere kombinierte Features eingesetzt: `rpm_diff = sensor_13_NRf_corrected_fan_speed - sensor_8_Nf_fan_speed` dient als robustes Mass für mechanische Abweichungen. `temp_to_pressure = sensor_4_T50_LPT_outlet_temp / sensor_11_Ps30_HPC_static_pres` quantifiziert potenzielle Effizienzverluste bei abweichenden Druckverhältnissen. `bleed_minus_temp = sensor_17_htBleed_bleed_enthalpy - sensor_4_T50_LPT_outlet_temp` identifiziert thermische Ungleichgewichte zwischen Zapfluft und Abgastemperatur, z. B. durch Leckagen.

##### Gruppe B: Treibstofffluss & Kühlung

Es wurde `sensor_12_phi_fuel_flow_per_Ps30` verwendet, da er die Brennstoffeffizienz relativ zum statischen Druck abbildet. Zusätzlich wurde `coolant_mean = (sensor_20_W31_HPT_coolant_bleed + sensor_21_W32_LPT_coolant_bleed)/2` gebildet, um den thermischen Kühlzustand robuster zu erfassen. Das kombinierte Feature `phi_to_bpr = sensor_12_phi_fuel_flow_per_Ps30 / sensor_15_BPR_bypass_ratio` erlaubt Rückschlüsse auf Effizienzverluste bei untypischer Strömungsverteilung.

##### Gruppe C: Mechanische Belastung

`sensor_14_NRc_corrected_core_speed` wurde als direkter Indikator für mechanische Belastung ausgewählt. Zusätzlich wurde `torque_ratio = sensor_14_NRc_corrected_core_speed / sensor_9_Nc_core_speed` gebildet, um die Lastverteilung normiert über Betriebspunkte hinweg zu erfassen. `sensor_9_Nc_core_speed` wurde aufgrund starker linearer Korrelation (r = 0.96) entfernt. Das gruppenübergreifende Feature `torque_times_bleed = sensor_14_NRc_corrected_core_speed * sensor_17_htBleed_bleed_enthalpy` verstärkt Zustände mit gleichzeitiger mechanischer und thermischer Belastung, wie sie bei ineffizientem Betrieb auftreten.

##### Gruppe D: Konstant / verrauscht

`sensor_6_P15_bypass_pres` wurde ausgeschlossen, da er über alle Units konstant bzw. stark verrauscht war und keinen Beitrag zur Zustandsdiagnose liefert.

##### Skalierung und Zeitnormalisierung

Alle Merkmale wurden anschliessend mithilfe von `standardize_by_op_cond()` standardisiert. Obwohl im Szenario **FD001** nur eine einzige Betriebsbedingung (`op_cond`-Gruppe) vorliegt, wurde die Funktion trotzdem verwendet, um ein konsistentes Vorgehen über alle Datensätze hinweg sicherzustellen. In diesem Fall entspricht die Skalierung einer **globalen Standardisierung** über alle Datenpunkte hinweg. Dies stellt sicher, dass alle Sensorwerte **dimensionslos und vergleichbar** sind, und verhindert, dass einzelne Sensoren aufgrund unterschiedlicher Wertebereiche **unverhältnismässig stark** ins Modell eingehen.

Zur Verbesserung der zeitlichen Einordnung wurde zudem die **Zykluszeit pro Unit auf den Bereich [0, 1] normiert** (`groupby("unit")["time"] / max`), um **Trends über den Lebensverlauf** unabhängig von der absoluten Lebensdauer sichtbar zu machen.

In [ ]:
df_train_01_fe = fe.add_combined_features(df_train_01_fe, dataset_name="FD001")
df_test_01_fe = fe.add_combined_features(df_test_01_fe, dataset_name="FD001")

df_train_01_fe, df_test_01_fe = util.standardize_by_op_cond(df_train_01_fe, df_test_01_fe)

df_train_01_fe["time"] = df_train_01_fe.groupby("unit")["time"].transform(lambda x: x / x.max())
df_test_01_fe["time"] = df_test_01_fe.groupby("unit")["time"].transform(lambda x: x / x.max())

df_train_01_fe = fe.select_columns(df_train_01_fe, [4, 11, 12, 14, 17], include_opsettings = False, dataset_name="FD001")
df_test_01_fe = fe.select_columns(df_test_01_fe, [4, 11, 12, 14, 17], include_opsettings = False, dataset_name="FD001")

df_train_01_fe

In [ ]:
# Reduktion nach Analyse
drop_cols = [
    "sensor_14_NRc_corrected_core_speed",
    "rpm_diff",
    "temp_to_pressure",
    "bleed_minus_temp",
    "torque_ratio",
    "torque_times_bleed"
]
df_train_01_fe.drop(columns=drop_cols, inplace=True)
df_test_01_fe.drop(columns=drop_cols, inplace=True)
# ---

toggle = util.make_toggle_shortcut(df_train_01_fe, "FD001_FE")

feature_cols = [col for col in df_train_01_fe.columns if col not in ["unit", "time", "RUL"]]

sensors_plots = [
    toggle("Korrelation", lambda df: eda.plot_sensor_correlation_matrix(df, sensor_cols=feature_cols, dataset_name="FD001_FE", annot=True)),
    toggle("Verteilung nach Lebensdauer", eda.plot_sensor_distributions_by_cycle_range, sensor_cols=feature_cols),
    toggle("Trend normierte Zeit", eda.plot_average_sensor_trend_normalized_time, sensor_cols=feature_cols),
    toggle("RUL-Korrelation", eda.plot_sensor_rul_correlation, sensor_cols=feature_cols),
]

sections = [
    util.make_dropdown_section(sensors_plots, "FD001_FE", use_cache=False),
]
tab_titles = ["Featureanalyse (FE)"]

eda_panel_FD001_FE = util.make_lazy_panel_with_tabs(
    sections,
    tab_titles=tab_titles,
    open_btn_text="FD001_FE öffnen",
    close_btn_text="Schliessen"
)
display(eda_panel_FD001_FE)

##### Reduktion nach Feature-Analyse

Nach der initialen Auswahl physikalisch sinnvoller und korrelierter Sensoren sowie der Berechnung mehrerer kombinierter Features wurde eine gezielte Reduktion der Featuremenge durchgeführt. Grundlage bildeten die normierten Zeitverläufe, Verteilungen nach Lebensdauer, Korrelationsmatrix sowie die Korrelation mit der Zielgrösse (RUL).

Folgende ursprünglich gewählte Merkmale wurden entfernt:

- `sensor_14_NRc_corrected_core_speed`, `torque_ratio`, `torque_times_bleed`: Zeigten entweder hohe Varianz im späteren Verlauf oder starke Korrelation untereinander bzw. mit bestehenden thermischen Merkmalen.
- `rpm_diff`, `temp_to_pressure`, `bleed_minus_temp`: Keine klaren Trends oder geringe Streuung, ohne nennenswerte Korrelation zur RUL.

Die finale Featuremenge enthält somit nur Merkmale mit stabilen zeitlichen Trends, guter Trennschärfe in der Lebensdauerverteilung und möglichst geringer Redundanz.

**Verwendete Sensoren:**

- **Direkt:** `sensor_4`, `sensor_11`, `sensor_12`, `sensor_17`  
- **Indirekt über kombinierte Features:** `sensor_15`, `sensor_20`, `sensor_21`

Die Gruppe C Mechanische Belastung wurde ausgeschlossen da die Sensoren `sensor_14` und `sensor_9` teils starke Varianz oder starke Redundanz zu bereits verwendeten thermischen Merkmalen zeigten. Trotz mehrerer getesteter Kombinationen (z. B. Verhältnis-, Differenz- und Log-Features) konnten keine robusten, verlässlich prädiktiven Trends identifiziert werden. Daher wurde Gruppe C in der finalen Featureauswahl nicht berücksichtigt.

##### Erweiterung durch temporale Fenstermerkmale

Um dynamische Informationen über den Degradationsverlauf zu erfassen, wurde die ursprüngliche Featuremenge mithilfe der Funktion `extract_temporal_features()` um aggregierte Merkmale aus mehreren Zeitfenstern erweitert. Für jeden Triebwerkslauf wurden um normierte Zeitpunkte (25 %, 50 %, 75 %) herum statistische Kennzahlen wie **Mittelwert, Standardabweichung, Min, Max, Range**, **lineare Regressionsslope und R²** sowie die **Mittelwertdifferenz** zwischen dem frühen und dem späten Bereich im Fenster berechnet. Dabei entspricht der frühe Bereich den **ersten 30 %** und der späte Bereich den **letzten 30 %** der Werte im jeweiligen Zeitfenster.

Das verwendete Fenster betrug jeweils **± 0.25 um den jeweiligen Zeitpunkt**, sodass **alle Sensorwerte pro Unit** in mindestens ein Fenster einflossen. Damit spiegeln die extrahierten Merkmale nicht nur lokale, sondern auch übergreifende Verläufe der Sensorzeitreihen wider.

Beim Testdatensatz kam eine angepasste Version (`extract_temporal_features_test()`) zum Einsatz. Dabei wurde pro Unit ein **einziges Zeitfenster um den letzten bekannten Zeitpunkt** (normierte Zeit = 1.0) betrachtet. Innerhalb dieses Fensters wurden **alle verfügbaren Sensorwerte ausgewertet**, um die aggregierten Merkmale zu berechnen. Die dabei entstehende Zielvariable (`RUL`) wurde anschliessend durch die **originalen Labelwerte** ersetzt, um eine faire Evaluation sicherzustellen und **Data-Leakage** zu vermeiden.

In [ ]:
df_train_01_fe = fe.extract_temporal_features(df_train_01_fe)
df_test_01_fe = fe.extract_temporal_features_test(df_test_01_fe)

df_train_01_fe

##### Feature Selection
Nach dem vollständigen Feature Engineering erfolgt nun die gezielte Reduktion der Merkmale mittels RandomForest-basierter Feature Selection. Ziel ist es, Merkmale mit sehr geringer Prädiktionsrelevanz zu identifizieren und zu entfernen, um Overfitting zu vermeiden und die Modellkomplexität zu reduzieren.

Dazu wird ein RandomForestRegressor auf den Trainingsdaten trainiert und die Feature-Wichtigkeiten (feature_importances_) berechnet. Alle Merkmale mit einer Wichtigkeit unterhalb eines definierten Schwellenwerts (< 0.005) werden verworfen.

Die wichtigsten Merkmale stammen überwiegend aus **temporalen Trendanalysen**, insbesondere aus Zeitreihenmerkmalen wie **Steigung (`slope`)**, **Regressionsgüte (`r²`)** und **Mittelwertunterschied (`mean_diff`)**. Dominant sind die Temperatur am LPT-Ausgang (`sensor_4_T50`) sowie kombinierte Effizienzkennzahlen wie `phi_to_bpr` und `coolant_mean`.  

Diese Features spiegeln **deutliche zeitliche Veränderungen** im Systemzustand wider und sind daher besonders gut geeignet, den Verschleissverlauf und die verbleibende Lebensdauer vorherzusagen.

In [ ]:
# === Vorbereitung der Daten ===
X_train = df_train_01_fe.drop(columns=["unit", "time_point", "RUL"])
y_train = df_train_01_fe["RUL"]
X_test = df_test_01_fe.drop(columns=["unit", "time_point", "RUL"])
y_test = df_test_01_fe["RUL"]

# Feature Selection durchführen
X_train, X_test, imp_df, drop_cols = fe.select_features(X_train, y_train, X_test)

# Top-10 Features anzeigen
imp_df.head(10)

#### 3.1.3 Modellierung

In diesem Schritt werden verschiedene Regressionsmodelle trainiert, um die verbleibende Nutzungsdauer (RUL) auf Basis der zuvor selektierten Features vorherzusagen. Ziel ist es, die leistungsfähigsten Modelle anhand praxisnaher Fehlerkennzahlen zu bewerten und das beste Modell für die weitere Verwendung auszuwählen.

##### Baseline-Modell

**Verwendete Bewertungsmetriken:**
- **RMSE (Root Mean Squared Error):** Durchschnittlicher quadratischer Fehler – misst die allgemeine Abweichung zwischen Vorhersage und Wahrheit.
- **R² (Bestimmtheitsmass):** Erklärt, wie viel Varianz der Zielvariable durch das Modell erklärt wird.
- **NASA-Score:** Eine speziell für RUL-Probleme entwickelte Metrik, die Fehler **asymmetrisch und exponentiell** bestraft.
  
Der NASA-Score wurde speziell für Remaining-Useful-Life-Prognosen entwickelt und berücksichtigt die unterschiedliche Schwere von Vorhersagefehlern:

- **Überschätzungen der RUL** (Modell zu optimistisch) werden **stark bestraft**, da sie in der Praxis zu unerwarteten Ausfällen führen können.  
- **Unterschätzungen** (Modell zu vorsichtig) sind weniger kritisch und werden entsprechend **milder bestraft**.

Die Strafe erfolgt exponentiell – mit unterschiedlicher Basis:

$$
\text{NASA-Score} =
\begin{cases}
\sum_{i=1}^{n} e^{- \frac{\hat{y}_i - y_i}{13}} - 1, & \text{falls } \hat{y}_i - y_i < 0 \\
\sum_{i=1}^{n} e^{\frac{\hat{y}_i - y_i}{10}} - 1, & \text{sonst}
\end{cases}
$$

Dabei ist $(\hat{y}_i) $ die vorhergesagte RUL und $(y_i)$ der wahre Wert.

In [ ]:
# Modellliste
models_list = models.get_model_list(weighted=False)

# Evaluation
results = []
for name, model in models_list:
    results.append(models.evaluate_model(model, X_train, y_train, X_test, y_test, model_name=name))
models_df = pd.DataFrame(results).sort_values("NASA-Score")
models_df

In [ ]:
models.plot_model_scores(
    models_df,
    score_col="NASA-Score",
    title="Modellvergleich: (Scores bis 2000 begrenzt)"
)

**Modellvergleich:**  
Die Abbildung zeigt den NASA-Score verschiedener Regressionsmodelle bei der Vorhersage der verbleibenden Nutzungsdauer (RUL). Je niedriger der Score, desto besser ist das Modell in Bezug auf sichere, realistische Vorhersagen.

- **Random Forest** erzielt den besten Score, gefolgt von **Gradient Boosting** und **Ridge**.  
- **Lineare Modelle** wie `Linear Regression` schneiden deutlich schlechter ab als regulierte Varianten.  
- Modelle wie `ElasticNet`, `Lasso`, `Decision Tree` und `KNN` erreichen den maximalen Score (2000) und gelten als ungeeignet für dieses Szenario.  
- Besonders **SVR** fällt durch seinen relativ hohen Fehler auf, obwohl es in anderen Kontexten oft gut performt.

Die starken Unterschiede zeigen, dass einfache lineare oder nicht regulierte Modelle den komplexen Degradationsverlauf nicht adäquat abbilden können. Ensembles wie Random Forest sind hier deutlich robuster.

In [ ]:
# Bestes Modell ungewichtet
best_model_name = models_df.iloc[0]["Model"]
model_map = {name: model for name, model in models_list}
best_model = model_map[best_model_name]
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

# Plot
models.plot_prediction_and_residuals(y_test, y_pred, model_name=best_model_name)

Der rechte Residuenplot zeigt, dass das Modell bei **niedriger RUL tendenziell zur Überschätzung** neigt – es prognostiziert also eine längere verbleibende Lebensdauer als tatsächlich vorhanden ist. Umgekehrt tritt bei hoher RUL häufig eine leichte Unterschätzung auf. Diese Tendenz ist insbesondere in späten Lebensphasen kritisch, da **Überschätzungen kurz vor dem Ausfall** das Risiko ungeplanter Stillstände deutlich erhöhen.

Der verwendete **NASA-Score berücksichtigt diese Asymmetrie** explizit, indem er Überschätzungen deutlich stärker bestraft als Unterschätzungen. Um diesem Umstand gezielt Rechnung zu tragen, wird im nächsten Schritt eine **gewichtete Modellbewertung** eingeführt:  
Beobachtungen mit geringer RUL erhalten ein höheres Gewicht, sodass das Modell stärker auf präzise Vorhersagen in sicherheitskritischen Phasen optimiert wird. Die Gewichtsfunktion nimmt dabei exponentiell mit der RUL ab:

$$
w(\text{RUL}) = 1 + 2 \cdot e^{- \frac{\text{RUL}}{25}}
$$

Die Parameter wurden so gewählt, dass die Gewichtung insbesondere im Bereich bis etwa **125 Zyklen** wirkt – also genau dort, wo im Residuenplot eine systematische Überschätzung auftritt. Damit erhalten Einheiten mit geringer Restlebensdauer ein bis zu dreifach höheres Gewicht. Die folgende Grafik visualisiert den Funktionsverlauf.

In [ ]:
# === Gewichtsfunktion definieren
rul_vals = np.linspace(0, 125, 200)
weights = 1 + 2 * np.exp(-rul_vals / 25)

plt.plot(rul_vals, weights)
plt.xlabel("RUL")
plt.ylabel("Gewicht")
plt.title("Gewichtsfunktion in Abhängigkeit vom RUL")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Gewichtete Modelle
weighted_models = models.get_model_list(weighted=True)
results_weighted = []
for name, model in weighted_models:
    results_weighted.append(models.evaluate_model_weighted(model, X_train, y_train, X_test, y_test, model_name=name))
models_df_weighted = pd.DataFrame(results_weighted)

# Kombinieren
models_df_combined = pd.concat([models_df, models_df_weighted]).sort_values("NASA-Score").reset_index(drop=True)
models_df_combined

In [ ]:
# Modellvergleichsplot
models.plot_model_scores(
    models_df,
    models_df_weighted,
    score_col="NASA-Score",
    title="Modellvergleich: vor und nach Gewichtung (Scores bis 2000 begrenzt)"
)

**Modellvergleich vor und nach Gewichtung:**

Die Abbildung zeigt den Vergleich der NASA-Scores für jedes Modell – jeweils **vor (schraffiert)** und **nach (vollflächig)** Anwendung der Gewichtung. Ziel war es, Überschätzungen bei niedriger RUL stärker zu bestrafen und so das Modellverhalten in sicherheitskritischen Phasen gezielter zu verbessern.

- Bei fast allen Modellen verbessert sich der Score leicht bis deutlich. Besonders ausgeprägt ist der Effekt bei **Linear Regression**, **Gradient Boosting** und **Ridge**.
- Bei **Random Forest** ist die Verbesserung zwar gering, aber vorhanden – das Modell war bereits ungewichtet sehr gut.
- Bei **ElasticNet** und **Lasso** bringt die Gewichtung zwar einen Fortschritt, allerdings verbleiben sie weiterhin im nicht praktikablen Bereich.
- **Decision Trees** bleiben unabhängig von der Gewichtung ungeeignet.

Insgesamt zeigt sich, dass eine gezielte Gewichtung der Trainingsdaten — abgestimmt auf die Schwere der Vorhersagefehler — zu einer robusteren Modellleistung führen kann, insbesondere bei Modellen, die empfindlich auf fehlerverteilte Daten reagieren.

In [ ]:
best = models_df_combined.iloc[0]
best_model_name = best["Model"]
base_name = best_model_name.replace(" (weighted)", "")
model_map_all = {**{name: model for name, model in weighted_models}, **{f"{name} (weighted)": model for name, model in weighted_models}}
best_model = model_map_all[best_model_name if best_model_name in model_map_all else base_name]

# Trainieren
if best_model_name.endswith("(weighted)"):
    weights = 1 + 2 * np.exp(-y_train / 25)
    best_model.fit(X_train, y_train, sample_weight=weights)
else:
    best_model.fit(X_train, y_train)

# Vorhersage + Plot
y_pred = best_model.predict(X_test)
models.plot_prediction_and_residuals(y_test, y_pred, model_name=best_model_name)

**Analyse des gewichteten Random Forest:**

Im Vergleich zur ungewichteten Variante zeigt der gewichtete Random Forest eine **verbesserte Modellanpassung in den kritischen RUL-Bereichen**. In der linken Grafik („Vorhersage vs. Wahrheit“) sind weniger starke Überschätzungen bei niedriger RUL sichtbar, was auf eine gezieltere Fehlerkontrolle in späten Lebensphasen hinweist.

Auch im Residuenplot (rechts) ist eine **leichte Zentrierung der Residuen im Bereich niedriger RUL** erkennbar. Die systematische Tendenz zur Überschätzung wurde abgemildert. Bei höherer RUL bleibt die Streuung vergleichbar zur ungewichteten Variante – hier hat die Gewichtung kaum Einfluss, was gewünscht ist.

Insgesamt bestätigt sich, dass die eingeführte Gewichtung ihre Wirkung entfaltet: **Fehler bei niedriger RUL – die im NASA-Score stark bestraft werden – wurden gezielt reduziert**, ohne das Verhalten in stabilen Bereichen negativ zu beeinflussen.

#### 3.1.4 Hyperparameter-Tuning

Nachdem die Modelle im vorherigen Schritt mit Standardparametern trainiert und bewertet wurden, erfolgt nun eine gezielte Optimierung der Hyperparameter in zwei aufeinander aufbauenden Stufen.

**Stufe 1 – Grobe Raster-Suche (GridSearch):**  
Zunächst wird für alle Modelle ein sinnvoller Parameterraum definiert und mittels `GridSearchCV` nach einer geeigneten Modellkonfiguration gesucht. Als Bewertungsmetrik kommt erneut der NASA-Score zum Einsatz. Diese erste Phase dient dazu, eine solide Ausgangsbasis zu schaffen und bereits klare Fehlkonfigurationen auszusortieren.

**Stufe 2 – Feinjustierung (RandomizedSearch):**  
Für eine Auswahl der vielversprechendsten Modelle aus der GridSearch wird im Anschluss eine umfassendere Feinabstimmung mit `RandomizedSearchCV` durchgeführt. Dabei werden grössere, kontinuierliche Hyperraumverteilungen verwendet, um durch stichprobenartige Suche bessere Parameterkombinationen zu finden.

In [ ]:
from sklearn.linear_model import (LinearRegression, Ridge, Lasso, ElasticNet)
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
# === Scorer vorbereiten ===
nasa_scorer = models.nasa_scorer

# === Modelle und GridSearch-Suchräume definieren ===
model_defs = [
    ("Linear Regression", LinearRegression(), True),
    ("Ridge", Ridge(), True),
    ("Lasso", Lasso(), True),
    ("ElasticNet", ElasticNet(), True),
    ("KNN", KNeighborsRegressor(), False),
    ("SVR", SVR(), False),
    ("Decision Tree", DecisionTreeRegressor(), True),
    ("Random Forest", RandomForestRegressor(), True),
    ("Gradient Boosting", GradientBoostingRegressor(), True),
]

param_grids = {
    "Linear Regression": {},
    "Ridge": {"alpha": [0.1, 1.0, 10.0]},
    "Lasso": {"alpha": [0.001, 0.01, 0.1]},
    "ElasticNet": {"alpha": [0.01, 0.1], "l1_ratio": [0.2, 0.5, 0.8]},
    "KNN": {"n_neighbors": [3, 5, 7]},
    "SVR": {"C": [1, 10], "epsilon": [0.1, 0.5]},
    "Decision Tree": {"max_depth": [4, 6, 10], "min_samples_leaf": [5, 10]},
    "Random Forest": {"n_estimators": [50, 100], "max_depth": [6, 10]},
    "Gradient Boosting": {"n_estimators": [50, 100], "learning_rate": [0.05, 0.1]},
}

# === GridSearchCV (erste HPT)
df_hpt_grid = models.run_grid_search(X_train, y_train, X_test, y_test, model_defs, param_grids)
df_hpt_grid

In [ ]:
rows_to_add = models_df[models_df["Model"].isin(["SVR", "KNN"])]
models_df_vor = pd.concat([models_df_weighted, rows_to_add], ignore_index=True)
models.plot_model_scores(
    models_df_vor,
    df_hpt_grid,
    score_col="NASA-Score",
    title="Modellvergleich: vor und nach HPT"
)

**Modellvergleich vor und nach erstem Hyperparameter-Tuning (HPT):**

Die Abbildung zeigt die Auswirkungen des Hyperparameter-Tunings (HPT) auf den NASA-Score der Modelle. Verglichen wird die Modellleistung **vor dem Tuning (schraffiert)** mit der Leistung **nach dem Tuning (vollflächig)**.

- Bei den meisten Modellen konnte der Score durch HPT weiter reduziert werden, insbesondere bei **Lasso**, **ElasticNet**, **Decision Tree** und **SVR**, die vorher relativ schlecht abschnitten.
- **Random Forest** und **Gradient Boosting** profitieren hingegen kaum vom HPT, da sie bereits zuvor mit guten Default-Werten arbeiteten.
- **KNN** bleibt weiterhin ungeeignet – trotz Tuning werden keine brauchbaren Scores erreicht.

Für das erweiterte Hyperparameter-Tuning mittels `RandomizedSearchCV` wurden bewusst nicht nur die leistungsstärksten Modelle ausgewählt, sondern gezielt solche, bei denen noch substantielles Optimierungspotenzial vermutet wurde:

- **Random Forest** und **Gradient Boosting** wurden weiter untersucht, da sie bereits starke Resultate liefern und eine Feinjustierung hier zusätzliche Leistungsgewinne ermöglichen kann.
- **Lasso**, **ElasticNet** und **SVR** zeigten in der GridSearch erkennbare Fortschritte und verfügen über komplexe Hyperparameter, deren breitere Variation durch Randomized Search zusätzliche Verbesserungen verspricht.
- **Decision Tree** wurde trotz signifikanter Verbesserung nicht weiter optimiert, da sein Beitrag im Vergleich zu den bereits berücksichtigten Ensemble-Methoden (**RF**, **GB**) begrenzt ist.
- **Ridge** und **Linear Regression** wurden ausgeschlossen, da sich ihre Performance im GridSearch-Schritt kaum verbessert hat. Im Fall der **Linear Regression** ist zudem **kein Hyperparameter** vorhanden, der überhaupt abgestimmt werden könnte.

Die Auswahl zielt darauf ab, **vielfältige Modellfamilien zu vertreten**, dabei jedoch Doppelungen zu vermeiden und Rechenressourcen auf aussichtsreiche Kandidaten zu konzentrieren.

In [ ]:
from scipy.stats import randint, uniform

# === Modelle + breite Parameterverteilungen
models_with_dists = {
    "Random Forest": (RandomForestRegressor(), {
        "n_estimators": randint(100, 500),
        "max_depth": [6, 10, None],
        "min_samples_leaf": randint(1, 6),
        "max_features": ["sqrt", "log2", None],
        "bootstrap": [True, False]
    }),
    "Gradient Boosting": (GradientBoostingRegressor(), {
        "n_estimators": randint(100, 500),
        "learning_rate": uniform(0.01, 0.2),
        "max_depth": [3, 6, 9],
        "subsample": uniform(0.6, 0.4),
        "min_samples_leaf": randint(1, 6)
    }),
    "Lasso": (Lasso(), {
        "alpha": uniform(0.0001, 0.1)
    }),
    "ElasticNet": (ElasticNet(), {
        "alpha": uniform(0.001, 0.1),
        "l1_ratio": uniform(0, 1)
    }),
    "SVR": (SVR(), {
        "C": uniform(1, 100),
        "epsilon": uniform(0.01, 1.0),
        "kernel": ["rbf", "linear", "poly"],
        "gamma": ["scale", "auto"]
    })
}

# === RandomizedSearchCV (zweite HPT)
X_train["unit"] = df_train_01_fe["unit"]
X_test["unit"] = df_test_01_fe["unit"]

df_hpt_rand = models.run_random_search(
    X_train.drop(columns=["unit"]),
    y_train,
    X_test.drop(columns=["unit"]),
    y_test,
    X_train["unit"],
    models_with_dists,
    n_iter=500
)

df_hpt_rand

In [ ]:
# === Hyperparameter-Spalten aus "Best Params" extrahieren
df_best_params = models.expand_best_params(df_hpt_rand )
df_best_params

In [ ]:
df_hpt_grid_red = df_hpt_grid[~df_hpt_grid["Model"].isin(["Linear Regression", "Ridge", "Decision Tree", "KNN"])]
df_plot = models.select_best_per_model(df_hpt_grid_red, df_hpt_rand)
models.plot_model_scores(
    df_hpt_grid_red,
    df_plot,
    score_col="NASA-Score",
    title="Modellvergleich: Vorher-Nachher nur für Modelle mit Randomized Search"
)

**Modellvergleich: Vorher–Nachher nur für Modelle mit Randomized Search**

Die Abbildung zeigt den gezielten Vergleich der Modellperformance **vor und nach dem erweiterten Hyperparameter-Tuning** mittels `RandomizedSearchCV`. Betrachtet wurden nur jene Modelle, die auf Basis der ersten Tuningrunde (GridSearch) als vielversprechend eingestuft wurden.

- **ElasticNet** profitiert am deutlichsten vom erweiterten Suchraum und kann seinen NASA-Score signifikant verbessern.
- Auch bei **Lasso** zeigt sich ein spürbarer Leistungsgewinn durch präzisere Abstimmung des Regularisierungsparameters.
- **Random Forest** und **Gradient Boosting** bleiben stabil – die Default-Parameter lagen bereits nahe am Optimum.
- **SVR** zeigt keinerlei Verbesserung – trotz erweitertem Hyperraum bleibt das Modell im Vergleich zur Konkurrenz deutlich zurück.

Insgesamt bestätigt sich, dass Randomized Search insbesondere bei **sensiblen, stark hyperparameterabhängigen Modellen** (wie Lasso/ElasticNet) lohnenswert ist, während robuste Modelle wie Random Forest oder Gradient Boosting kaum profitieren.

#### 3.1.5 Evaluation

Nach Abschluss der zweistufigen Hyperparameter-Optimierung folgt nun die abschliessende Bewertung der besten Modellkonfigurationen. Für jedes Modell, das in der Randomized-Search-Phase ein überzeugendes Ergebnis erzielt hat, wird die finale Variante erneut instanziiert, mit gewichteter Verlustfunktion auf den Trainingsdaten trainiert und auf den Testdaten evaluiert.

Bewertet wird anhand des **NASA-Scores**, der auch in dieser finalen Phase als zentrales Auswahlkriterium dient.

Anschliessend wird das **beste Modell** anhand des minimalen NASA-Scores ausgewählt und dessen Vorhersageverhalten visuell analysiert.

In [ ]:
results = []
y_preds = []
models_list = []

model_map = {
    "Random Forest": RandomForestRegressor,
    "Gradient Boosting": GradientBoostingRegressor,
    "Lasso": Lasso,
    "ElasticNet": ElasticNet,
    "SVR": SVR,
}

for _, row in df_plot.iterrows():
    name = row["Model"]
    params = row["Best Params"]
    model_cls = model_map[name]

    model = model_cls(**params)
    eval_result = models.evaluate_model_weighted(model, X_train, y_train, X_test, y_test, model_name=name)

    results.append(eval_result)
    y_preds.append(model.predict(X_test))
    models_list.append(model)

df_test_eval = pd.DataFrame(results)
best_idx = df_test_eval["NASA-Score"].idxmin()
df_test_eval = df_test_eval.loc[[best_idx]].reset_index(drop=True)

y_pred_best = y_preds[best_idx]
y_test_best = y_test
best_model_name = df_test_eval.loc[0, "Model"]
df_test_eval

In [ ]:
models.plot_prediction_and_residuals(y_test_best, y_pred_best, best_model_name)

**Abschliessende Evaluation des besten Modells: Random Forest (weighted)**

Das finale Modell, ein **gewichteter Random Forest**, erzielte den besten NASA-Score aller getesteten Modelle:

- **RMSE-Test:** 17.79  
- **R²-Test:** 0.82  
- **NASA-Score:** 470.81

Die Vorhersagegrafik (links) zeigt eine gute Übereinstimmung zwischen vorhergesagter und tatsächlicher RUL, insbesondere im mittleren Bereich. Nur in sehr hohen RUL-Bereichen treten kleinere systematische Abweichungen auf, was für dieses Szenario tolerierbar ist.

Im **Residuenplot** (rechts) ist die Fehlerverteilung relativ gleichmässig um Null zentriert – ohne starke systematische Verzerrung. Damit erfüllt das Modell nicht nur die Anforderungen an Vorhersagegüte, sondern auch an **robuste Fehlerverteilung** über verschiedene RUL-Bereiche hinweg.

Insgesamt lässt sich festhalten:  
Der gewichtete Random Forest bildet den komplexen Degradationsverlauf zuverlässig ab und vermeidet gleichzeitig kritische Überschätzungen in späten Lebensphasen – was insbesondere durch den niedrigen NASA-Score bestätigt wird.

##### **Fazit zu Kapitel 3.1 – Analyse für FD001**

Die Analyse des einfachsten C-MAPSS-Szenarios FD001 hat gezeigt, dass auch unter konstanten Betriebsbedingungen eine präzise RUL-Vorhersage anspruchsvoll bleibt. Durch gezielte Feature-Auswahl, temporale Merkmalsextraktion sowie eine schrittweise Modelloptimierung konnte jedoch ein leistungsfähiges Vorhersagemodell entwickelt werden.

Besonders hervorgetan hat sich der **gewichtete Random Forest**, der dank robuster Struktur und Fehlergewichtung sowohl in klassischen Metriken (RMSE, R²) als auch im NASA-Score überzeugte. Die Gewichtung erwies sich dabei als entscheidender Faktor zur Reduktion systematischer Überschätzungen bei niedriger RUL.

Damit bildet FD001 eine solide Grundlage, auf der sich die Generalisierbarkeit und Belastbarkeit der entwickelten Pipeline in komplexeren Szenarien (Kapitel 3.2–3.4) überprüfen lässt.

---
### 3.2 Analyse für FD002

Der Datensatz FD002 enthält variable Betriebsbedingungen bei gleichzeitig einem einzigen Degradationsmodus. Im Vergleich zu FD001 ist das Verhalten der Sensoren komplexer, da die Betriebspunkte schwanken.  
Die Analyse dieses Szenarios liefert wichtige Erkenntnisse darüber, wie stark sich Sensorwerte durch die Operation Settings beeinflussen lassen und welche Normalisierungen oder Features erforderlich sind.

#### 3.2.1 Explorative Datenanalyse (EDA)


In [ ]:
if EDA_FD002:
    # ─────────────────────────────
    # Konfiguration für FD002 EDA
    # ─────────────────────────────
    # Steuerung des PNG-Cachings pro Plot-Sektion.
    # Falls False: PNG wird gelöscht und neu erstellt.
    USE_CACHE = {
        "overview": True,
        "ops": True,
        "sensors": True,
        "cluster": True,
    }
    FORCE_RECOMPUTE_TSNE_DBSCAN = FORCE_RECOMPUTE_TSNE_DBSCAN
    # FORCE_RECOMPUTE_TSNE_DBSCAN = True
    # ─────────────────────────────

    toggle = util.make_toggle_shortcut(df_train_02, "FD002")
    sensor_cols = [f"sensor_{i}" for i in list(range(1, 18)) + [19, 20, 21]]

    overview_plots = [
        toggle("1-1. Lebensdauerkennzahlen", eda.describe_life_stats),
        toggle("1-2. Lebensdauerverteilung", eda.plot_life_distribution),
    ]
    ops_plots = [
        toggle("2-1. Verläufe Operation Settings", eda.plot_opsetting_curves, unit_ids=[244, 112]),
        toggle("2-2. Korrelation Operation Settings", eda.plot_opsetting_correlation_matrix),
        toggle("2-3. Verteilung im letzten Zyklus", eda.plot_opsetting_box_violin_last_cycle),
        toggle("2-4. Verteilung nach Quantilen", eda.plot_opsetting_distributions_by_cycle_range, lower_quantile=0.25, upper_quantile=0.75),
        toggle("2-5. RUL-Korrelation", eda.plot_opsetting_rul_correlation),
        toggle("2-6. Trend normierte Zeit", eda.plot_average_opsetting_trend_normalized_time),
    ]
    sensors_plots = [
        toggle("3-1. Sensorverläufe", eda.plot_single_sensor_curves, unit_ids=range(1, 6), rolling_window=10),
        toggle("3-2. Sensor-Overlay", eda.plot_sensor_overlay, unit_id=112, dataset_name="FD002-112"),
        toggle("3-3. Sensor-Korrelation", lambda df: (
            (fig := plt.figure(figsize=(28, 10))),
            (axs := fig.subplots(1, 2)),
            eda.plot_sensor_correlation_matrix(df, dataset_name="FD002", ax=axs[0]),
            eda.plot_sensor_correlation_matrix(df, sensor_cols=sensor_cols, dataset_name="FD002 (ohne konstante)", annot=True, ax=axs[1]),
            plt.tight_layout(),
        )),
        toggle("3-4. Box/Violin letzter Zyklus", eda.plot_sensor_box_violin_last_cycle, sensor_cols=sensor_cols),
        toggle("3-5. Sensorverteilung nach Lebensdauer", eda.plot_sensor_distributions_by_cycle_range, sensor_cols=sensor_cols),
        toggle("3-6. Sensorverteilungen nach op_cond", eda.plot_sensor_distributions_by_cycle_range, hue_col="op_cond", sensor_cols=sensor_cols),
        toggle("3-7. RUL-Korrelation Sensoren", eda.plot_sensor_rul_correlation, sensor_cols=sensor_cols),
        toggle("3-8. Sensortrend normierte Zeit", eda.plot_average_sensor_trend_normalized_time, sensor_cols=sensor_cols),
    ]

    # Einstellungen von plot_tsne_dbscan_clusters
    toggle_tsne = widgets.Output()
    with toggle_tsne:
        fig, labels = eda.plot_tsne_dbscan_clusters(
            df_train_02,
            feature_cols = sensor_cols,
            dataset_name="FD002",
            dbscan_eps = 2.5,
            force_recompute=FORCE_RECOMPUTE_TSNE_DBSCAN
        )
        df_train_02["cluster_tsne"] = labels

    cluster_plots = [
        toggle("4-1. TSNE + DBSCAN Cluster", eda.plot_tsne_dbscan_clusters), # plot_tsne_dbscan_clusters zeigt nur den Plot an
        toggle("4-2. op_settings je Cluster (Boxplot)", eda.plot_op_settings_vs_cluster, cluster_col="cluster_tsne"),
        toggle("4-3. Cluster-Transitions (Sankey)", lambda df: (
            fig := eda.plot_cluster_transitions_sankey(df, cluster_col="cluster_tsne", dataset_name="FD002"),
            fig.update_layout(width=1800, height=600),
            fig
        )),
        toggle("4-4. Durchschnittlicher Zeitpunkt je Cluster", eda.plot_cluster_average_time, cluster_col="cluster_tsne", dataset_name="FD002"),
        toggle("4-5. Clusterverteilung letzter Zyklus", eda.plot_cluster_distribution_last_cycle, cluster_col="cluster_tsne"),
        toggle("4-6. Lebensdauer pro finalem Cluster", eda.plot_lifetime_boxplot_by_cluster, cluster_col="cluster_tsne"),
        toggle("4-7. Mittlere Sensorwerte pro Cluster", eda.plot_mean_normalized_sensors_by_cluster, sensor_cols=sensor_cols, cluster_col="cluster_tsne"),
        toggle("4-8. Sensorverteilungen nach Cluster", eda.plot_sensor_distributions_by_cycle_range, hue_col="cluster_tsne", sensor_cols=sensor_cols),
        toggle("4-9. Trend Sensoren je Cluster",
               lambda df: util.make_cluster_navigation_panel(
                   df=df,
                   cluster_col="cluster_tsne",
                   cluster_plot_func=eda.plot_average_sensor_trend_normalized_time,
                   sensor_cols=sensor_cols,
                   dataset_name="FD002",
                   force_recompute=not USE_CACHE["cluster"]
               )),
        toggle("4-10. Cluster-Zusammenfassung (Tabelle)", eda.summarize_cluster_characteristics, cluster_col="cluster_tsne"),
    ]

    if PRECOMPUTE_ALL_PLOTS:
        util.cache_util.cache_all_plots(
            [overview_plots, ops_plots, sensors_plots, cluster_plots],
            dataset_name="FD002",
            force_recompute=FORCE_RECOMPUTE_PLOTS
        )

    sections = [
        util.make_dropdown_section(overview_plots, "FD002", use_cache=USE_CACHE["overview"]),
        util.make_dropdown_section(ops_plots, "FD002", use_cache=USE_CACHE["ops"]),
        util.make_dropdown_section(sensors_plots, "FD002", use_cache=USE_CACHE["sensors"]),
        util.make_dropdown_section(cluster_plots, "FD002", use_cache=USE_CACHE["cluster"]),
    ]
    tab_titles = [
        "1. Übersicht",
        "2. Operation Settings",
        "3. Sensoren",
        "4. Clusteranalyse",
    ]
    eda_panel_FD002 = util.make_lazy_panel_with_tabs(
        sections,
        tab_titles=tab_titles,
        open_btn_text="FD002 EDA öffnen",
        close_btn_text="Schliessen"
    )
    display(eda_panel_FD002)

#### EDA-Zusammenfassung FD002  
Der Datensatz FD002 weist variierende Betriebsbedingungen auf, wobei `op_setting_1` und `op_setting_2` stark schwanken und hoch korreliert sind. Die Triebwerke zeigen erneut eine rechtsschiefe Lebensdauerverteilung mit moderater Streuung.  

Die Operation Settings beeinflussen mehrere Sensorverteilungen deutlich, haben aber nur eine geringe direkte Korrelation zur RUL. Die Sensoren sind stark untereinander korreliert, was auf eine hohe Redundanz hinweist. Einzelne Sensoren wie `sensor_14`, `15` und `16` liefern dennoch potenziell prädiktive Informationen.

Die t-SNE/DBSCAN-Clusteranalyse identifiziert sieben Lebenszustände:
- darunter **Cluster 3** als instabiler Zwischenzustand,  
- sowie **Cluster 6** als möglicher finaler Zustand kurz vor Ausfall.

Diese Erkenntnisse bilden die Grundlage für das Feature Engineering und die spätere Modellierung. Eine vollständige Detailauswertung befindet sich in **Anhang B.2**.

#### 3.2.2 Feature Engineering

#### 3.2.3 Modellierung

#### 3.2.4 Hyperparameter-Tuning

#### 3.2.5 Evaluation

---
### 3.3 Analyse für FD003
FD003 weist konstante Betriebsbedingungen, aber mehrere Degradationsmodi auf. Damit eignet sich der Datensatz gut, um rein sensorbasierte Unterschiede im Degradationsverhalten zu untersuchen, ohne dass sich zusätzlich die Betriebszustände verändern.

#### 3.3.1 Explorative Datenanalyse (EDA)

In [ ]:
if EDA_FD003:
    # ─────────────────────────────
    # Konfiguration für FD003 EDA
    # ─────────────────────────────
    # Steuerung des PNG-Cachings pro Plot-Sektion.
    # Falls False: PNG wird gelöscht und neu erstellt.
    USE_CACHE = {
        "overview": True,
        "ops": True,
        "sensors": True,
        "cluster": True,
    }
    FORCE_RECOMPUTE_TSNE_DBSCAN = FORCE_RECOMPUTE_TSNE_DBSCAN
    # FORCE_RECOMPUTE_TSNE_DBSCAN = True
    # ─────────────────────────────

    toggle = util.make_toggle_shortcut(df_train_03, "FD003")
    sensor_cols = [f"sensor_{i}" for i in [2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 20, 21]]

    overview_plots = [
        toggle("1-1. Lebensdauerkennzahlen", eda.describe_life_stats),
        toggle("1-2. Lebensdauerverteilung", eda.plot_life_distribution),
    ]
    ops_plots = [
        toggle("2-1. Verläufe Operation Settings", eda.plot_opsetting_curves, unit_ids=[99, 55]),
        toggle("2-2. Korrelation Operation Settings", eda.plot_opsetting_correlation_matrix),
        toggle("2-3. Verteilung im letzten Zyklus", eda.plot_opsetting_box_violin_last_cycle),
        toggle("2-4. Verteilung nach Quantilen", eda.plot_opsetting_distributions_by_cycle_range, lower_quantile=0.25, upper_quantile=0.75),
        toggle("2-5. RUL-Korrelation", eda.plot_opsetting_rul_correlation),
        toggle("2-6. Trend normierte Zeit", eda.plot_average_opsetting_trend_normalized_time),
    ]
    sensors_plots = [
        toggle("3-1. Sensorverläufe", eda.plot_single_sensor_curves, unit_ids=range(1, 6), rolling_window=10),
        toggle("3-2. Sensor-Overlay", eda.plot_sensor_overlay, unit_id=55, dataset_name="FD003-55"),
        toggle("3-3. Sensor-Korrelation", lambda df: (
            (fig := plt.figure(figsize=(28, 10))),
            (axs := fig.subplots(1, 2)),
            eda.plot_sensor_correlation_matrix(df, dataset_name="FD003", ax=axs[0]),
            eda.plot_sensor_correlation_matrix(df, sensor_cols=sensor_cols, dataset_name="FD003 (ohne konstante)", annot=True, ax=axs[1]),
            plt.tight_layout(),
        )),
        toggle("3-4. Box/Violin letzter Zyklus", eda.plot_sensor_box_violin_last_cycle, sensor_cols=sensor_cols),
        toggle("3-5. Sensorverteilung nach Lebensdauer", eda.plot_sensor_distributions_by_cycle_range, sensor_cols=sensor_cols),
        toggle("3-6. Sensorverteilungen nach op_cond", eda.plot_sensor_distributions_by_cycle_range, hue_col="op_cond", sensor_cols=sensor_cols),
        toggle("3-7. RUL-Korrelation Sensoren", eda.plot_sensor_rul_correlation, sensor_cols=sensor_cols),
        toggle("3-8. Sensortrend normierte Zeit", eda.plot_average_sensor_trend_normalized_time, sensor_cols=sensor_cols),
    ]

    # Einstellungen von plot_tsne_dbscan_clusters
    toggle_tsne = widgets.Output()
    with toggle_tsne:
        fig, labels = eda.plot_tsne_dbscan_clusters(
            df_train_03,
            feature_cols = sensor_cols,
            dataset_name="FD003",
            dbscan_eps = 3,
            force_recompute=FORCE_RECOMPUTE_TSNE_DBSCAN
        )
        df_train_03["cluster_tsne"] = labels

    cluster_plots = [
        toggle("4-1. TSNE + DBSCAN Cluster", eda.plot_tsne_dbscan_clusters), # plot_tsne_dbscan_clusters zeigt nur den Plot an
        toggle("4-2. op_settings je Cluster (Boxplot)", eda.plot_op_settings_vs_cluster, cluster_col="cluster_tsne"),
        toggle("4-3. Cluster-Transitions (Sankey)", lambda df: (
            fig := eda.plot_cluster_transitions_sankey(df, cluster_col="cluster_tsne", dataset_name="FD003"),
            fig.update_layout(width=1800, height=600),
            fig
        )),
        toggle("4-4. Durchschnittlicher Zeitpunkt je Cluster", eda.plot_cluster_average_time, cluster_col="cluster_tsne", dataset_name="FD003"),
        toggle("4-5. Clusterverteilung letzter Zyklus", eda.plot_cluster_distribution_last_cycle, cluster_col="cluster_tsne"),
        toggle("4-6. Lebensdauer pro finalem Cluster", eda.plot_lifetime_boxplot_by_cluster, cluster_col="cluster_tsne"),
        toggle("4-7. Mittlere Sensorwerte pro Cluster", eda.plot_mean_normalized_sensors_by_cluster, sensor_cols=sensor_cols, cluster_col="cluster_tsne"),
        toggle("4-8. Sensorverteilungen nach Cluster", eda.plot_sensor_distributions_by_cycle_range, hue_col="cluster_tsne", sensor_cols=sensor_cols),
        toggle("4-9. Trend Sensoren je Cluster",
               lambda df: util.make_cluster_navigation_panel(
                   df=df,
                   cluster_col="cluster_tsne",
                   cluster_plot_func=eda.plot_average_sensor_trend_normalized_time,
                   sensor_cols=sensor_cols,
                   dataset_name="FD003",
                   force_recompute=not USE_CACHE["cluster"]
               )),
        toggle("4-10. Cluster-Zusammenfassung (Tabelle)", eda.summarize_cluster_characteristics, cluster_col="cluster_tsne"),
    ]

    if PRECOMPUTE_ALL_PLOTS:
        util.cache_util.cache_all_plots(
            [overview_plots, ops_plots, sensors_plots, cluster_plots],
            dataset_name="FD003",
            force_recompute=FORCE_RECOMPUTE_PLOTS
        )

    sections = [
        util.make_dropdown_section(overview_plots, "FD003", use_cache=USE_CACHE["overview"]),
        util.make_dropdown_section(ops_plots, "FD003", use_cache=USE_CACHE["ops"]),
        util.make_dropdown_section(sensors_plots, "FD003", use_cache=USE_CACHE["sensors"]),
        util.make_dropdown_section(cluster_plots, "FD003", use_cache=USE_CACHE["cluster"]),
    ]
    tab_titles = [
        "1. Übersicht",
        "2. Operation Settings",
        "3. Sensoren",
        "4. Clusteranalyse",
    ]
    eda_panel_FD003 = util.make_lazy_panel_with_tabs(
        sections,
        tab_titles=tab_titles,
        open_btn_text="FD003 EDA öffnen",
        close_btn_text="Schliessen"
    )
    display(eda_panel_FD003)

#### EDA-Zusammenfassung FD003  
Der Datensatz FD003 enthält konstante Betriebspunkte bei mehreren Triebwerktypen und weist eine hohe Streuung in der Lebensdauer auf. Die Lebensdauerverteilung ist erneut rechtsschief, jedoch mit deutlich grösserer Varianz als bei FD001/FD002.

Die Operation Settings bleiben über den gesamten Lebenszyklus konstant und zeigen keinerlei signifikanten Einfluss auf die Sensoren oder die RUL.

Die Sensoren zeigen hingegen ausgeprägte Trends und Strukturen. Mehrere Sensoren korrelieren deutlich mit der Lebensdauer, andere besitzen multimodale Verteilungen oder bilden stark korrelierte Gruppen. Diese Merkmale bieten gute Voraussetzungen für Feature Engineering.

Die Clusteranalyse mit t-SNE/DBSCAN identifiziert fünf Betriebszustände:  
- **Cluster 0** dominiert als stabiler Nominalzustand,  
- **Cluster 1, 2 und 4** zeigen unterschiedliche Degradationsphasen,  
- **Cluster 3** ist kurzzeitig zu beginn aktiv, instabil und wird als Übergangszustand interpretiert.

Diese Struktur liefert eine gute Grundlage für zustandsbasierte Modellierungsansätze. Die vollständige Detailanalyse befindet sich in **Anhang B.3**.


#### 3.3.2 Feature Engineering


#### 3.3.3 Modellierung


#### 3.3.4 Hyperparameter-Tuning


#### 3.3.5 Evaluation

---
### 3.4 Analyse für FD004
FD004 kombiniert die komplexesten Bedingungen aller C-MAPSS-Datensätze: variable Betriebspunkte und mehrere Degradationsmodi.
Dieser Datensatz stellt somit das realistischste, aber auch herausforderndste Szenario dar. Ziel der Analyse ist es, robuste Muster trotz starker Streuung und Rauschen zu identifizieren.


#### 3.4.1 Explorative Datenanalyse (EDA)

In [ ]:
if EDA_FD004:
    # ─────────────────────────────
    # Konfiguration für FD004 EDA
    # ─────────────────────────────
    # Steuerung des PNG-Cachings pro Plot-Sektion.
    # Falls False: PNG wird gelöscht und neu erstellt.
    USE_CACHE = {
        "overview": True,
        "ops": True,
        "sensors": True,
        "cluster": True,
    }
    FORCE_RECOMPUTE_TSNE_DBSCAN = FORCE_RECOMPUTE_TSNE_DBSCAN
    # FORCE_RECOMPUTE_TSNE_DBSCAN = True
    # ─────────────────────────────

    toggle = util.make_toggle_shortcut(df_train_04, "FD004")
    sensor_cols = [f"sensor_{i}" for i in range(1, 22)]

    overview_plots = [
        toggle("1-1. Lebensdauerkennzahlen", eda.describe_life_stats),
        toggle("1-2. Lebensdauerverteilung", eda.plot_life_distribution),
    ]
    ops_plots = [
        toggle("2-1. Verläufe Operation Settings", eda.plot_opsetting_curves, unit_ids=[214, 118]),
        toggle("2-2. Korrelation Operation Settings", eda.plot_opsetting_correlation_matrix),
        toggle("2-3. Verteilung im letzten Zyklus", eda.plot_opsetting_box_violin_last_cycle),
        toggle("2-4. Verteilung nach Quantilen", eda.plot_opsetting_distributions_by_cycle_range, lower_quantile=0.25, upper_quantile=0.75),
        toggle("2-5. RUL-Korrelation", eda.plot_opsetting_rul_correlation),
        toggle("2-6. Trend normierte Zeit", eda.plot_average_opsetting_trend_normalized_time),
    ]
    sensors_plots = [
        toggle("3-1. Sensorverläufe", eda.plot_single_sensor_curves, unit_ids=range(1, 6), rolling_window=10),
        toggle("3-2. Sensor-Overlay", eda.plot_sensor_overlay, unit_id=118, dataset_name="FD004-118"),
        toggle("3-3. Sensor-Korrelation", lambda df: (
            (fig := plt.figure(figsize=(28, 10))),
            (axs := fig.subplots(1, 2)),
            eda.plot_sensor_correlation_matrix(df, dataset_name="FD004", ax=axs[0]),
            eda.plot_sensor_correlation_matrix(df, sensor_cols=sensor_cols, dataset_name="FD004 (ohne konstante)", annot=True, ax=axs[1]),
            plt.tight_layout(),
        )),
        toggle("3-4. Box/Violin letzter Zyklus", eda.plot_sensor_box_violin_last_cycle, sensor_cols=sensor_cols),
        toggle("3-5. Sensorverteilung nach Lebensdauer", eda.plot_sensor_distributions_by_cycle_range, sensor_cols=sensor_cols),
        toggle("3-6. Sensorverteilungen nach op_cond", eda.plot_sensor_distributions_by_cycle_range, hue_col="op_cond", sensor_cols=sensor_cols),
        toggle("3-7. RUL-Korrelation Sensoren", eda.plot_sensor_rul_correlation, sensor_cols=sensor_cols),
        toggle("3-8. Sensortrend normierte Zeit", eda.plot_average_sensor_trend_normalized_time, sensor_cols=sensor_cols),
    ]

    # Einstellungen von plot_tsne_dbscan_clusters
    toggle_tsne = widgets.Output()
    with toggle_tsne:
        fig, labels = eda.plot_tsne_dbscan_clusters(
            df_train_04,
            feature_cols = sensor_cols,
            dataset_name="FD004",
            dbscan_eps = 3,
            force_recompute=FORCE_RECOMPUTE_TSNE_DBSCAN
        )
        df_train_04["cluster_tsne"] = labels

    cluster_plots = [
        toggle("4-1. TSNE + DBSCAN Cluster", eda.plot_tsne_dbscan_clusters), # plot_tsne_dbscan_clusters zeigt nur den Plot an
        toggle("4-2. op_settings je Cluster (Boxplot)", eda.plot_op_settings_vs_cluster, cluster_col="cluster_tsne"),
        toggle("4-3. Cluster-Transitions (Sankey)", lambda df: (
            fig := eda.plot_cluster_transitions_sankey(df, cluster_col="cluster_tsne", dataset_name="FD004"),
            fig.update_layout(width=1800, height=600),
            fig
        )),
        toggle("4-4. Durchschnittlicher Zeitpunkt je Cluster", eda.plot_cluster_average_time, cluster_col="cluster_tsne", dataset_name="FD004"),
        toggle("4-5. Clusterverteilung letzter Zyklus", eda.plot_cluster_distribution_last_cycle, cluster_col="cluster_tsne"),
        toggle("4-6. Lebensdauer pro finalem Cluster", eda.plot_lifetime_boxplot_by_cluster, cluster_col="cluster_tsne"),
        toggle("4-7. Mittlere Sensorwerte pro Cluster", eda.plot_mean_normalized_sensors_by_cluster, sensor_cols=sensor_cols, cluster_col="cluster_tsne"),
        toggle("4-8. Sensorverteilungen nach Cluster", eda.plot_sensor_distributions_by_cycle_range, hue_col="cluster_tsne", sensor_cols=sensor_cols),
        toggle("4-9. Trend Sensoren je Cluster",
               lambda df: util.make_cluster_navigation_panel(
                   df=df,
                   cluster_col="cluster_tsne",
                   cluster_plot_func=eda.plot_average_sensor_trend_normalized_time,
                   sensor_cols=sensor_cols,
                   dataset_name="FD004",
                   force_recompute=not USE_CACHE["cluster"]
               )),
        toggle("4-10. Cluster-Zusammenfassung (Tabelle)", eda.summarize_cluster_characteristics, cluster_col="cluster_tsne"),
    ]

    if PRECOMPUTE_ALL_PLOTS:
        util.cache_util.cache_all_plots(
            [overview_plots, ops_plots, sensors_plots, cluster_plots],
            dataset_name="FD004",
            force_recompute=FORCE_RECOMPUTE_PLOTS
        )

    sections = [
        util.make_dropdown_section(overview_plots, "FD004", use_cache=USE_CACHE["overview"]),
        util.make_dropdown_section(ops_plots, "FD004", use_cache=USE_CACHE["ops"]),
        util.make_dropdown_section(sensors_plots, "FD004", use_cache=USE_CACHE["sensors"]),
        util.make_dropdown_section(cluster_plots, "FD004", use_cache=USE_CACHE["cluster"]),
    ]
    tab_titles = [
        "1. Übersicht",
        "2. Operation Settings",
        "3. Sensoren",
        "4. Clusteranalyse",
    ]
    eda_panel_FD004 = util.make_lazy_panel_with_tabs(
        sections,
        tab_titles=tab_titles,
        open_btn_text="FD004 EDA öffnen",
        close_btn_text="Schliessen"
    )
    display(eda_panel_FD004)

#### EDA-Zusammenfassung FD004  
Der Datensatz FD004 kombiniert mehrere Triebwerkstypen mit stark schwankenden Betriebspunkten und langer Lebensdauer. Die Lebensdauerverteilung ist dabei ebenfalls rechtsschief, mit ähnlich grosser Streuung wie in FD003. Die Operation Settings 1 und 2 variieren stark und beeinflussen viele Sensorverteilungen deutlich, während Setting 3 konstant bleibt und vernachlässigt werden kann.

Die Sensorverläufe sind oft verrauscht und zeigen hohe Redundanz, mit wenigen klaren RUL-Trends. Daher ist gezieltes Preprocessing entscheidend.

Die t-SNE/DBSCAN-Clusteranalyse liefert eine gut interpretierbare Struktur mit 18 Zuständen:
- **Cluster 0–9**: Früh- bzw. Zwischenzustände mit teils instabilen Verläufen,  
- **Cluster 10–17**: Klar erkennbare Degradationsphasen am Lebensende.

FD004 stellt hohe Anforderungen an Feature Engineering und Modellierung, bietet aber gleichzeitig Potenzial für robuste, zustandsadaptive Modelle. Eine vollständige Detailauswertung befindet sich in **Anhang B.4**.


#### 3.4.2 Feature Engineering


#### 3.4.3 Modellierung


#### 3.4.4 Hyperparameter-Tuning


#### 3.4.5 Evaluation

---
## 4. Kombinierte Analyse (FD001–FD004)


### 4.1 Explorative Analyse


### 4.2 Feature Engineering


### 4.3 Modellierung


### 4.4 Evaluation

---
## 5. Zusammenfassung und Ausblick


### 5.1 Ergebnisse im Vergleich


### 5.2 Lessons Learned


### 5.3 Weiterführende Ideen

---
---

## Anhang A – Erklärung der verwendeten Plots


Zur besseren Nachvollziehbarkeit der Explorativen Datenanalyse (EDA) werden im Folgenden die wichtigsten Plot-Typen kurz erläutert. Die Beschreibungen beziehen sich nicht auf konkrete Ergebnisse, sondern auf den allgemeinen Zweck und die Lesart der Visualisierungen.

### 1. Lebensdauerkennzahlen  
**Typ:** Tabellenausgabe + Histogramm  
Zentrale Kennzahlen wie Median, Mittelwert, Min/Max und Standardabweichung der Lebensdauer sowie deren Verteilung (z. B. rechtsschief bei Run-to-Failure-Daten).

### 2. Verläufe Operation Settings  
**Typ:** Liniendiagramme einzelner Units  
Visualisiert den zeitlichen Verlauf der Betriebspunkte (`op_setting_1–3`) über den Lebenszyklus. Beurteilt Konstanz oder Variation der Betriebsbedingungen.

### 3. Korrelation zu RUL (Settings/Sensoren)  
**Typ:** Balkendiagramm  
Korrelationskoeffizienten zwischen aktuellen Featurewerten (z. B. im letzten Zyklus) und der Restlebensdauer (RUL). Dient zur Identifikation prädiktiver Merkmale.

### 4. Sensorverläufe einzelner Units  
**Typ:** Liniendiagramme  
Zeigt typische Sensorverläufe über den Lebenszyklus einzelner Triebwerke. Hilft, Trends oder Auffälligkeiten visuell zu erkennen.

### 5. Überlagerte Sensortrends  
**Typ:** Liniendiagramm mit vielen Kurven  
Mehrere Sensorverläufe einer Unit überlagert dargestellt. Dient zur Interpretation von globalen Trends.

### 6. Verteilungen (nach Lebensdauer, Cluster, Settings)  
**Typ:** Box- und Violinplots  
Vergleicht Sensorwerte im letzten Zyklus oder in Lebensdauer-Quantilen – gruppiert nach Cluster oder Betriebspunkt-Kategorie. Zeigt Unterschiede und Streuungen zwischen Gruppen.

### 7. Korrelationsmatrizen  
**Typ:** Heatmaps  
Visualisieren lineare Zusammenhänge zwischen Features (Sensoren/Settings) über den gesamten Datensatz. Unterstützt die Auswahl nicht-redundanter Merkmale.

### 8. Trends über normierte Zeit  
**Typ:** Liniendiagramm (aggregiert)  
Zeigt den mittleren Verlauf eines Features über normierte Zeit (0 = Beginn, 1 = Ausfall). Dient zur Identifikation typischer Degradationstrends.

### 9. t-SNE + DBSCAN  
**Typ:** 2D-Scatterplot  
Dimensionalitätsreduktion und Clustering. Visualisiert unterschiedliche Betriebs- oder Degradationszustände über den gesamten Lebenszyklus hinweg.

### 10. Sankey-Diagramm (Clustertransitions)  
**Typ:** Flussdiagramm  
Zeigt, wie Units im Lebensverlauf von einem Clusterzustand in einen anderen wechseln. Hilft bei der Interpretation typischer Zustandsverläufe.